#**Pré-Processamento**

In [ ]:
!pip install tensorflow
!pip install keras

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
import tensorflow

from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.tree import DecisionTreeRegressor

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import optimizers

from matplotlib.lines import Line2D

import random
import os
import re

# Configuração visual dos gráficos
plt.style.use("fivethirtyeight")
color_pal = sns.color_palette()

# Fixar seed para reprodutibilidade
def set_seed(seed=42):
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    random.seed(seed)
    tensorflow.random.set_seed(seed)

set_seed(42)

In [ ]:
# Carrega o dataset.
# O arquivo "ToyDataset.xlsx" deve estar dentro da pasta "Data"
# no mesmo diretório do notebook.

df = pd.read_excel(
    "Data/ToyDataset.xlsx",
    parse_dates=['DATA']
)

In [ ]:
intervalos_ferias = [
    (pd.to_datetime('2023-01-01'), pd.to_datetime('2023-04-02')),  # Férias antes do primeiro período letivo de 2023
    (pd.to_datetime('2023-07-23'), pd.to_datetime('2023-08-09')),  # Férias entre os períodos letivos de 2023
    (pd.to_datetime('2023-12-24'), pd.to_datetime('2023-12-31')),  # Férias depois do segundo período letivo de 2023
    (pd.to_datetime('2024-01-01'), pd.to_datetime('2024-03-17')),  # Férias antes do primeiro período letivo de 2024
    (pd.to_datetime('2024-07-21'), pd.to_datetime('2024-08-11')),  # Férias entre os períodos letivos de 2024
    (pd.to_datetime('2024-12-15'), pd.to_datetime('2024-12-31'))   # Férias depois do segundo período letivo de 2024
]

# Função para verificar se a data está no intervalo de férias
def verificar_ferias(data, intervalos):
    for inicio, fim in intervalos:
        if inicio <= data <= fim:
            return 1  # Férias
    return 0  # Não é férias

# Atualizar o DataFrame com a nova coluna "Férias"
df['É_FÉRIAS'] = df['DATA'].apply(lambda x: verificar_ferias(x, intervalos_ferias))

In [ ]:
feriados = pd.to_datetime([
    '2023-01-01',  # Ano Novo
    '2023-02-20',  # Carnaval (Facultativo)
    '2023-02-21',  # Carnaval (Facultativo)
    '2023-02-22',  # Quarta-feira de Cinzas (Facultativo)
    '2023-02-23',  # Recesso
    '2023-02-24',  # Recesso
    '2023-02-27',  # Recesso
    '2023-02-28',  # Recesso
    '2023-04-07',  # Sexta-Feira Santa
    '2023-04-21',  # Tiradentes
    '2023-04-23',  # São Jorge (Feriado Municipal)
    '2023-05-01',  # Dia do Trabalho
    '2023-06-08',  # Corpus Christi (Facultativo)
    '2023-06-24',  # Feriado Municipal
    '2023-07-29',  # Feriado Municipal
    '2023-09-07',  # Independência do Brasil
    '2023-10-12',  # Nossa Senhora Aparecida
    '2023-10-15',  # Dia do Professor (Facultativo)
    '2023-10-16',  # Dia do Comerciário (Feriado Municipal)
    '2023-10-28',  # Dia do Servidor Público (Facultativo)
    '2023-11-02',  # Dia de Finados
    '2023-11-15',  # Proclamação da República
    '2023-11-20',  # Dia da Consciência Negra (Feriado Estadual)
    '2023-12-25',  # Natal
    '2024-01-01',  # Ano Novo
    '2024-02-12',  # Carnaval (Facultativo)
    '2024-02-13',  # Carnaval (Facultativo)
    '2024-02-14',  # Quarta-feira de Cinzas (Facultativo)
    '2024-03-29',  # Sexta-Feira Santa
    '2024-04-21',  # Tiradentes
    '2024-04-23',  # São Jorge (Feriado Estadual)
    '2024-05-01',  # Dia do Trabalhador
    '2024-05-30',  # Corpus Christi (Facultativo)
    '2024-06-24',  # Dia de São João Batista (Feriado Municipal)
    '2024-07-29',  # Aniversário de Macaé (Feriado Municipal)
    '2024-09-07',  # Independência do Brasil
    '2024-10-12',  # Nossa Senhora Aparecida
    '2024-10-28',  # Dia do Servidor Público (Facultativo)
    '2024-11-02',  # Dia de Finados
    '2024-11-15',  # Proclamação da República
    '2024-11-20',  # Dia da Consciência Negra
    '2024-12-25',  # Natal
])

df['FERIADO'] = df['DATA'].isin(feriados).astype(int)
df['PRÉ_FERIADO'] = df['DATA'].isin(feriados - pd.Timedelta(days=1)).astype(int)
df['PÓS_FERIADO'] = df['DATA'].isin(feriados + pd.Timedelta(days=1)).astype(int)

df.loc[df['FERIADO'] == 1, 'POLO_QUANTIDADE'] = 0

In [ ]:
def transformar_dados(df, usar_ciclico=True):
    df = df.copy()

    # Manter apenas dias úteis (segunda a sexta)
    df = df.loc[df['DATA'].dt.weekday < 5]

    # Features temporais básicas
    df['DIA_SEMANA'] = df['DATA'].dt.weekday
    df['MES'] = df['DATA'].dt.month

    if usar_ciclico:
        df['DIA_SEM_SIN'] = np.sin(2 * np.pi * df['DIA_SEMANA'] / 5)
        df['DIA_SEM_COS'] = np.cos(2 * np.pi * df['DIA_SEMANA'] / 5)
        df['MES_SIN'] = np.sin(2 * np.pi * df['MES'] / 12)
        df['MES_COS'] = np.cos(2 * np.pi * df['MES'] / 12)

        df.drop(columns=['DIA_SEMANA', 'MES'], inplace=True)

    return df

In [ ]:
# Ajuste conforme o experimento:
# True: aplica encoding cíclico (sen/cos)
# False: mantém DIA_SEMANA e MES como valores inteiros
planilha = transformar_dados(df, usar_ciclico=True)

In [ ]:
def processar_pratos(planilha, usar_pratos=True):

    # Registrar quais pratos originais foram convertidos em cada categoria
    historico = {
        'sem_serviço': [],
        'prato_nao_informado': [],
        'aves': [],
        'aves_cremosas': [],
        'bovino_ensopadas': [],
        'bovino_cremosas': [],
        'suino': [],
        'peixe_fruto_do_mar': [],
        'mistos': [],
        'outros': [],
        'desconhecido': []
    }

    if not usar_pratos:
        # Remove colunas relacionadas ao cardápio quando não forem usadas no experimento
        colunas_prato = [col for col in planilha.columns if 'PRATO_PRINCIPAL' in col or col.startswith('prato_')]
        planilha = planilha.drop(columns=colunas_prato, errors='ignore')
        return planilha, historico

    # Manter coluna original temporariamente para histórico
    planilha['PRATO_PRINCIPAL_ORIGINAL'] = planilha['PRATO_PRINCIPAL']

    # Categorias e palavras-chave para classificação dos pratos
    categorias = {
        'sem_serviço': ['feriado', 'recesso', 'final do contrato', 'sem serviço'],
        'prato_nao_informado': ['prato não informado'],
        'peixe_fruto_do_mar': ['peixe', 'camarão', 'salmão', 'tilápia'],
        'suino': ['lombo', 'suíno', 'mignon suíno', 'lombinho'],
        'mistos': ['misto', 'churrasquinho misto', 'goulash misto', 'picadinho misto'],
        'aves_cremosas': ['fricassê', 'fricasse', 'estrogonofe de frango', 'empadão de frango', 'lasanha de frango'],
        'bovino_cremosas': ['estrogonofe de carne', 'lasanha à bolonhesa', 'lasanha bolonhesa', 'lasanha de carne'],
        'aves': ['frango', 'sobrecoxa', 'galinha'],
        'bovino_ensopadas': ['carne', 'picadinho de carne', 'almôndega', 'almodega', 'goulash', 'escalope de carne']
    }
    # Classificar prato em categorias usando palavras-chave
    def classificar_prato(nome_prato):
        original = nome_prato

        if pd.isna(nome_prato):
            historico['desconhecido'].append(original)
            return 'desconhecido'

        nome_prato_proc = nome_prato.lower().strip()

        for categoria, palavras in categorias.items():
            for palavra in palavras:
                if re.search(rf'\b{re.escape(palavra)}\b', nome_prato_proc):
                    historico[categoria].append(original)
                    return categoria

        historico['outros'].append(original)
        return 'outros'

    # Aplicar classificação
    planilha['PRATO_PRINCIPAL'] = planilha['PRATO_PRINCIPAL'].astype(str).apply(classificar_prato)

    # Visualização rápida da distribuição das categorias
    tabela_final = planilha['PRATO_PRINCIPAL'].value_counts().reset_index()
    tabela_final.columns = ['Categoria', 'Quantidade']
    tabela_final.index = tabela_final.index + 1
    display(tabela_final)

    # One-hot encoding das categorias de prato
    planilha_one_hot = pd.get_dummies(planilha['PRATO_PRINCIPAL'], prefix='prato').astype(int)
    planilha = pd.concat([planilha, planilha_one_hot], axis=1)

    # Remover colunas originais de prato antes da modelagem
    planilha = planilha.drop(columns=['PRATO_PRINCIPAL', 'PRATO_PRINCIPAL_ORIGINAL'], errors='ignore')

    return planilha, historico

In [ ]:
# Ajuste conforme o experimento:
# usar_pratos=True: aplica categorização, one-hot encoding e remove coluna original
# usar_pratos=False: remove coluna de pratos (sem usar cardápio na previsão)
df, historico = processar_pratos(planilha, usar_pratos=True)

In [ ]:
# Ajustes finais do dataframe para modelagem e visualização
df = df.set_index('DATA')
df = df.astype(np.float32)

df['POLO_QUANTIDADE'].plot(style=".", figsize=(15,5), color=color_pal[0],
                           title="Consumo Diário de Refeições")

In [ ]:
# Verificar valores zerados e NaN
indices_zero = df[df['POLO_QUANTIDADE'] == 0].index
print(indices_zero)

nan_indices = df[df['POLO_QUANTIDADE'].isna()].index
print(nan_indices)

In [ ]:
# Garantir consistência dos dados: substituir possíveis NaN por 0 (não encontrados no recorte atual)
df['POLO_QUANTIDADE'] = df['POLO_QUANTIDADE'].fillna(0)

In [ ]:
df['POLO_QUANTIDADE'].plot(
    style=".",
    figsize=(15,5),
    color=color_pal[0],
    title="Consumo Diário de Refeições"
)

##**Modelos Baseados em Árvores de Decisão**

In [ ]:
df1 = df.copy()

In [ ]:
# Número de dias passados a considerar para lag
n_passado = 5

# Criar colunas de lag com valores anteriores
for i in range(1, n_passado + 1):
    df1[f'POLO_QUANTIDADE_{i}'] = df1['POLO_QUANTIDADE'].shift(i)

# Salvar datas removidas por causa do lag
datas_removidas_lag = df1[df1.isna().any(axis=1)].index

# Remover linhas com NaN geradas pelo lag
df1 = df1.dropna()

In [ ]:
# Proporção de treino
proporcao_treino = 0.8

# Calcular número de amostras para treino
tamanho_treino = int(len(df1) * proporcao_treino)

# Data que separa treino e teste
data_corte_treino = df1.index[tamanho_treino]

# Divisão temporal
treino = df1[df1.index < data_corte_treino]
teste = df1[df1.index >= data_corte_treino]

In [ ]:
# Salvar as datas removidas do conjunto de teste
datas_removidas_teste = teste.iloc[:n_passado].index

#Remover os primeiros 5 dias do conjunto de teste
teste = teste.iloc[n_passado:]

In [ ]:
df1.columns

In [ ]:
alvo = 'POLO_QUANTIDADE'

# Seleciona todas as colunas, exceto o alvo
features = [col for col in df1.columns if col != alvo]

X = df1[features]
y = df1[[alvo]]

In [ ]:
print("Features selecionadas:")
print(features)

In [ ]:
X_treino = treino[features]
y_treino = treino[alvo]

X_teste = teste[features]
y_teste = teste[alvo]

**EXECUTE APENAS A CÉLULA DO ALGORITMO QUE DESEJA UTILIZAR**

In [ ]:
#VERSÃO ÁRVORE

# Inicializa regressor
regressor = DecisionTreeRegressor(random_state=42)

# Grid de hiperparâmetros
param_grid = {
    "max_depth": [3, 4, 5, 6, 7, 8, 10, 12],
    "min_samples_split": [2, 5, 10, 15, 20],
    "min_samples_leaf": [1, 2, 4, 6, 8],
    "max_features": [None, "sqrt", "log2"],
    "criterion": ["squared_error", "absolute_error"]
}

# Busca pelos melhores hiperparâmetros via cross-validation
search = GridSearchCV(
    regressor,
    param_grid,
    cv=5,
    scoring='neg_root_mean_squared_error'
).fit(X_treino.to_numpy(), y_treino.to_numpy())

print("Os melhores hiperparâmetros são ", search.best_params_)

# Salva o melhor modelo para uso posterior
reg = search.best_estimator_

In [ ]:
#Versão XGBOOST

# Classe personalizada para evitar previsões negativas
class XGBRegressorPositivo(BaseEstimator, RegressorMixin):
    def __init__(self, **kwargs):
        self.model = xgb.XGBRegressor(**kwargs)

    def fit(self, X, y):
        self.model.fit(X, y)
        return self

    def predict(self, X):
        preds = self.model.predict(X)
        return np.maximum(preds, 0)  # Corrige valores negativos

    def get_params(self, deep=True):
        return self.model.get_params(deep)

    def set_params(self, **params):
        self.model.set_params(**params)
        return self

# Inicializa regressor XGBoost personalizado
regressor = XGBRegressorPositivo(eval_metric='rmse', random_state=42)

# Grid de hiperparâmetros
param_grid = {
    "max_depth": [3, 4, 5, 6, 7, 8],
    "n_estimators": [300, 400, 500, 600, 700],
    "learning_rate": [0.015, 0.020, 0.025, 0.05],
    "base_score": [0.5],
    "booster": ['gbtree'],
    "objective": ['reg:squarederror'],
}

# Busca pelos melhores hiperparâmetros
search = GridSearchCV(regressor, param_grid, cv=5).fit(X_treino.to_numpy(), y_treino.to_numpy())
print("Os melhores hiperparâmetros são ", search.best_params_)

# Salva o melhor modelo para uso posterior
reg = search.best_estimator_

In [ ]:
treino = treino.copy()
teste = teste.copy()

# Criar previsões usando .to_numpy() para evitar UserWarning de feature names
treino["Previsão"] = reg.predict(X_treino.to_numpy())
teste["Previsão"] = reg.predict(X_teste.to_numpy())

# Criar o gráfico
fig, ax = plt.subplots(figsize=(15, 5))

# Definir fundo branco
ax.set_facecolor('white')
fig.patch.set_facecolor('white')

# Plotar valores reais e previsões
treino["POLO_QUANTIDADE"].plot(ax=ax, label="Treino - Real", color='blue')
treino["Previsão"].plot(ax=ax, label="Treino - Prev.", color='orange', linestyle='--')
teste["POLO_QUANTIDADE"].plot(ax=ax, label="Teste - Real", color='skyblue')
teste["Previsão"].plot(ax=ax, label="Teste - Prev.", color='red', linestyle='--')

# Remover título e label do eixo X
ax.set_title("")
ax.set_xlabel("")

# Posicionar legenda abaixo do gráfico
ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.2), ncol=2)

# Ajustar layout e exibir
plt.tight_layout()
plt.show()

In [ ]:
# Calcular métricas de treino e teste
rmse_treino = np.sqrt(mean_squared_error(treino["POLO_QUANTIDADE"], treino["Previsão"]))
r2_treino = r2_score(treino["POLO_QUANTIDADE"], treino["Previsão"])
rmse_teste = np.sqrt(mean_squared_error(teste["POLO_QUANTIDADE"], teste["Previsão"]))
r2_teste = r2_score(teste["POLO_QUANTIDADE"], teste["Previsão"])

# Criar a tabela de métricas e formatar valores
tabela_metricas = pd.DataFrame({
    "Conjunto": ["Treino", "Teste"],
    "RMSE": [rmse_treino, rmse_teste],
    "R²": [r2_treino, r2_teste]
})
tabela_metricas["RMSE"] = tabela_metricas["RMSE"].apply(lambda x: f"{x:.5f}")
tabela_metricas["R²"] = tabela_metricas["R²"].apply(lambda x: f"{x:.5f}")

# Criar a figura da tabela e ajustar visual
fig, ax = plt.subplots(figsize=(5, 1.8))
ax.axis('off')  # Remover eixos
tabela = ax.table(
    cellText=tabela_metricas.values,
    colLabels=tabela_metricas.columns,
    cellLoc='center',
    colColours=['#4682B4'] * len(tabela_metricas.columns),
    loc='center'
)
tabela.auto_set_font_size(False)
tabela.set_fontsize(10)
tabela.auto_set_column_width([0, 1, 2])
plt.subplots_adjust(left=0.05, right=0.95, top=0.95, bottom=0.05)  # Ajustar layout

##**Modelo LSTM**

In [ ]:
# Criar cópia do DataFrame para LSTM
df2 = df.copy()
df2.shape

In [ ]:
# Dividir em treino (64%), validação (16%) e teste (20%)
tamanho_treino = round(len(df2) * 0.64)
tamanho_validacao = round(len(df2) * 0.16)

df_para_treinamento = df2[:tamanho_treino]
df_para_validacao = df2[tamanho_treino:tamanho_treino + tamanho_validacao]
df_para_teste = df2[tamanho_treino + tamanho_validacao:]

df_para_treinamento.shape, df_para_validacao.shape, df_para_teste.shape

In [ ]:
# Definir coluna alvo do LSTM
coluna_alvo_lstm = ['POLO_QUANTIDADE']

colunas_para_escalonar = coluna_alvo_lstm
colunas_nao_escalonadas = [col for col in df2.columns if col not in coluna_alvo_lstm]

In [ ]:
# Criar o escalonador
escalador = MinMaxScaler(feature_range=(0, 1))

# Aplicar escalonamento no conjunto de treino
df_para_treinamento_scaled = df_para_treinamento.copy()
df_para_treinamento_scaled[colunas_para_escalonar] = escalador.fit_transform(
    df_para_treinamento[colunas_para_escalonar]
)

# Aplicar escalonamento no conjunto de validação
df_para_validacao_scaled = df_para_validacao.copy()
df_para_validacao_scaled[colunas_para_escalonar] = escalador.transform(
    df_para_validacao[colunas_para_escalonar]
)

# Aplicar escalonamento no conjunto de teste
df_para_teste_scaled = df_para_teste.copy()
df_para_teste_scaled[colunas_para_escalonar] = escalador.transform(
    df_para_teste[colunas_para_escalonar]
)


In [ ]:
# Concatenar colunas escalonadas e não escalonadas
df_para_treinamento_final = pd.concat(
    [df_para_treinamento_scaled[colunas_para_escalonar], df_para_treinamento[colunas_nao_escalonadas]],
    axis=1
)

df_para_validacao_final = pd.concat(
    [df_para_validacao_scaled[colunas_para_escalonar], df_para_validacao[colunas_nao_escalonadas]],
    axis=1
)

df_para_teste_final = pd.concat(
    [df_para_teste_scaled[colunas_para_escalonar], df_para_teste[colunas_nao_escalonadas]],
    axis=1
)

In [ ]:
def criar_XY(conjunto_dados, n_passado):
    conjunto_dados = conjunto_dados.values  # Converter para array NumPy
    entradas_X = []  # Armazenar entradas
    saidas_Y = []    # Armazenar saídas (alvo)

    for i in range(n_passado, len(conjunto_dados)):
        entradas_X.append(conjunto_dados[i - n_passado:i, :])  # Selecionar todas as colunas
        saidas_Y.append(conjunto_dados[i, 0])  # O alvo é a primeira coluna (POLO_QUANTIDADE)

    return np.array(entradas_X), np.array(saidas_Y)

In [ ]:
# Definir número de dias passados a considerar
n_passado = 5

# Criar os conjuntos X e Y
treino_X, treino_Y = criar_XY(df_para_treinamento_final, n_passado)
validacao_X, validacao_Y = criar_XY(df_para_validacao_final, n_passado)
teste_X, teste_Y = criar_XY(df_para_teste_final, n_passado)

In [ ]:
# Treinar e avaliar LSTM
def treinar_e_avaliar_modelo(learning_rate, units, batch_size, epochs):
    adam = optimizers.Adam(learning_rate=learning_rate)

    # Definir modelo LSTM
    model_lstm = Sequential()

    # Escolher a arquitetura:
    # 1 camada LSTM + Dense (ativa por padrão)
    model_lstm.add(LSTM(units, activation='relu', input_shape=(treino_X.shape[1], treino_X.shape[2])))

    # 2 camadas LSTM + Dense (descomentar para testar)
    # model_lstm.add(LSTM(units, activation='relu', input_shape=(treino_X.shape[1], treino_X.shape[2]), return_sequences=True))
    # model_lstm.add(LSTM(units, activation='relu'))

    model_lstm.add(Dense(1, activation='relu'))
    model_lstm.compile(loss='mse', optimizer=adam)

    # Definir EarlyStopping
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    # Treinar modelo
    model_lstm.fit(
        treino_X, treino_Y,
        validation_data=(validacao_X, validacao_Y),
        epochs=epochs, batch_size=batch_size, verbose=2,
        callbacks=[early_stopping]
    )

    # Avaliar no conjunto de validação
    predicoes_validacao = model_lstm.predict(validacao_X)

    # Reverter escalonamento
    predicoes_unscaled = escalador.inverse_transform(predicoes_validacao.reshape(-1, 1))
    validacao_Y_unscaled = escalador.inverse_transform(validacao_Y.reshape(-1, 1))

    # Calcular RMSE
    rmse = np.sqrt(mean_squared_error(validacao_Y_unscaled, predicoes_unscaled))

    print(f'Parâmetros: learning_rate={learning_rate}, units={units}, batch_size={batch_size}')
    print(f'RMSE (Validação): {rmse}')

    return model_lstm, rmse

In [ ]:
resultados = []

# Hiperparâmetros a testar
learning_rates = [0.005, 0.01, 0.025, 0.05]
units_list = [5, 10, 20, 25]
batch_sizes = [4, 8, 16]

best_rmse = float('inf')
best_model = None

# Loop de experimentos
for lr in learning_rates:
    for units in units_list:
        for batch in batch_sizes:
            set_seed(42)
            modelo, rmse = treinar_e_avaliar_modelo(lr, units, batch, epochs=500)

            resultados.append({
                'learning_rate': lr,
                'units': units,
                'batch_size': batch,
                'rmse': rmse
            })

            # Atualizar melhor modelo
            if rmse < best_rmse:
                best_rmse = rmse
                best_model = modelo

print(f'\nMelhor modelo encontrado com RMSE: {best_rmse}')

In [ ]:
# Criar função para fazer previsões e calcular métricas
def fazer_previsoes_e_calcular_metricas(modelo, X, Y, escalador, indice_ajustado, n_passado):
    # Fazer previsões
    previsao = modelo.predict(X)
    previsao_unscaled = escalador.inverse_transform(previsao.reshape(-1, 1))
    Y_unscaled = escalador.inverse_transform(Y.reshape(-1, 1))

    # Calcular métricas
    rmse = np.sqrt(mean_squared_error(Y_unscaled, previsao_unscaled))
    r2 = r2_score(Y_unscaled, previsao_unscaled)

    # Ajustar índice considerando janela
    indice_ajustado = indice_ajustado[n_passado:]

    return previsao_unscaled, Y_unscaled, rmse, r2, indice_ajustado

In [ ]:
# Definir janela de tempo
n_passado = 5

# Treino
treino_pred, treino_real, rmse_treino, r2_treino, indice_treino_ajustado = fazer_previsoes_e_calcular_metricas(
    best_model, treino_X, treino_Y, escalador, df_para_treinamento_final.index, n_passado
)

# Validação
validacao_pred, validacao_real, rmse_validacao, r2_validacao, indice_validacao_ajustado = fazer_previsoes_e_calcular_metricas(
    best_model, validacao_X, validacao_Y, escalador, df_para_validacao_final.index, n_passado
)

# Teste
teste_pred, teste_real, rmse_teste, r2_teste, indice_teste_ajustado = fazer_previsoes_e_calcular_metricas(
    best_model, teste_X, teste_Y, escalador, df_para_teste_final.index, n_passado
)

In [ ]:
# Criar tabela de métricas
tabela_metricas = pd.DataFrame({
    "Conjunto": ["Treino", "Validação", "Teste"],
    "RMSE": [rmse_treino, rmse_validacao, rmse_teste],
    "R²": [r2_treino, r2_validacao, r2_teste]
})

# Formatar valores com 2 casas decimais
tabela_metricas["RMSE"] = tabela_metricas["RMSE"].apply(lambda x: f"{x:.5f}")
tabela_metricas["R²"] = tabela_metricas["R²"].apply(lambda x: f"{x:.5f}")

# Criar figura e tabela
fig, ax = plt.subplots(figsize=(5, 2))
ax.axis('off')
tabela = ax.table(cellText=tabela_metricas.values,
                  colLabels=tabela_metricas.columns,
                  cellLoc='center',
                  colColours=['#4682B4'] * len(tabela_metricas.columns),
                  loc='center')

# Ajustar tamanho da fonte e colunas
tabela.auto_set_font_size(False)
tabela.set_fontsize(10)
tabela.auto_set_column_width([0, 1, 2])

# Ajustar layout da figura
plt.subplots_adjust(left=0.05, right=0.95, top=0.95, bottom=0.05)

# Exibir tabela
plt.show()

In [ ]:
# Criar figura e eixo
fig, ax = plt.subplots(figsize=(15, 5))
ax.set_facecolor('white')
fig.patch.set_facecolor('white')

# Plotar valores reais
ax.plot(indice_treino_ajustado, treino_real, color='blue', linestyle='-')
ax.plot(indice_validacao_ajustado, validacao_real, color='skyblue', linestyle='-')
ax.plot(indice_teste_ajustado, teste_real, color='green', linestyle='-')

# Plotar previsões
ax.plot(indice_treino_ajustado, treino_pred, color='orange', linestyle='--')
ax.plot(indice_validacao_ajustado, validacao_pred, color='red', linestyle='--')
ax.plot(indice_teste_ajustado, teste_pred, color='purple', linestyle='--')

# Remover título e label do eixo X
ax.set_title("")
ax.set_xlabel("")

# Criar legenda personalizada
legendas_personalizadas = [
    Line2D([0], [0], color='blue', linestyle='-', label='Treino - Valor Real'),
    Line2D([0], [0], color='orange', linestyle='--', label='Treino - Previsão'),
    Line2D([0], [0], color='skyblue', linestyle='-', label='Validação - Valor Real'),
    Line2D([0], [0], color='red', linestyle='--', label='Validação - Previsão'),
    Line2D([0], [0], color='green', linestyle='-', label='Teste - Valor Real'),
    Line2D([0], [0], color='purple', linestyle='--', label='Teste - Previsão'),
]

# Posicionar legenda em 3 colunas abaixo do gráfico
ax.legend(handles=legendas_personalizadas, loc='upper center', bbox_to_anchor=(0.5, -0.35),
          ncol=3, frameon=False)

# Ajustar layout e exibir gráfico
plt.tight_layout()
plt.show()
